## Тренировочные упражнения по теме _"Подзапросы"_

### Описание задачи
Ноутбук содержит решение тренировочных задач с использованием коррелированных и некоррелированных подзапросов в SELECT, FROM, JOIN, WHERE и ORDER BY.  

### Дано: *5 таблиц*

#### 1. Таблица *titles*:

|Название атрибута |	Тип атрибута | Смысловое значение | 
|:---|:---:|:---|
|id				|int	|ID сериала/фильма |
|genres			|text	|Жанр |
|original_title	|text	|Оригинальное название|
|popular_title	|text	|Популярное название|
|is_adult		|bool	|Метка 18+|
|type			|text	|Вид _(i.e. фильм, сериал, короткометражка)_|
|run_time		|int	|Продолжительность|
|year_of_start	|int	|Год первого выпуска |
|year_of_end	|int	|Год последнего выпуска|

#### 2. Таблица *episodes*:

|Название атрибута |	Тип атрибута | Смысловое значение | 
|:---|:---:|:---|
|title_id		|int	|ID серии |
|parent_id		|int	|ID сериала, которому принадлежит серия |
|season			|int	|Номер сезона|
|episode		|int	|Номер серии|

#### 3. Таблица *ratings*:

|Название атрибута	|Тип атрибута	|Смысловое значение|
|:---|:---:|:---|
|title_id		|int	|ID сериала/фильма |
|avg_rating		|numeric|Средний рейтинг |
|vote_cnt		|int	|Число отданных голосов|

#### 4. Таблица *principals*:

|Название атрибута	|Тип атрибута	|Смысловое значение|
|:---|:---:|:---|
|title_id		|int	|ID сериала/фильма |
|ordering		|int	|Номер в списке каста  |
|person_id		|int	|ID члена каста |
|category		|text	|Категория члена каста *(i.e. actor, writer etc)* |
|job			|text	|Должность члена каста *(i.e. writer, producer etc)*|
|characters		|text	|Имя сыгранного персонажа |

#### 5. Таблица *persons*:
|Название атрибута	|Тип атрибута	|Смысловое значение|
|:---|:---:|:---|
|id				|int	|ID персоналии |
|name			|text	|Имя персоналии  |
|professions	|text	|Должность персоналии |
|year_of_birth	|int	|Год рождения |
|year_of_death	|int	|Год смерти|


### Тренировочная задача 1: 
Напишите SQL-запрос, который выведет названия *(name)* сериалов, у которых первый эпизод первого сезона получил рейтинг 10. Назовите результирующее поле *name*. Результат отсортируйте в лексикографическом порядке.

```
SELECT original_title AS name
  FROM titles
 WHERE id IN (
              SELECT ep.parent_id
                FROM episodes AS ep
                JOIN ratings AS r ON r.title_id=ep.title_id
               WHERE ep.season = 1
                 AND ep.episode = 1
                 AND r.avg_rating = 10
              )
 ORDER BY name ASC;
 
```
**Итоговый вывод**: *2 rows*


|name|
|:---|
|If Star Wars Wasn't Serious|
|The Dentist Answers You|

***

### Тренировочная задача 2: 
Напишите SQL-запрос, который выведет оригинальное название *(original_title)*, год начала *(year_of_start)* и количество эпизодов произведений, у которых больше 500 серий, **отсортировав их по убыванию количества серий**. Для этого используйте подзапрос к таблице *episodes* с группировкой по полю *parent_id*. Выведите только строки, которые есть в таблице *titles* и в сгруппированной таблице.

Назовите поле с количеством эпизодов произведений *ep_cnt*. Отсортируйте результат по этому полю в порядке убывания.

```
SELECT t.original_title, 
       t.year_of_start,
       epi500.ep_cnt
  FROM titles AS t
  JOIN (
        SELECT epi.parent_id,
               COUNT(1) AS ep_cnt
          FROM episodes AS epi       
         GROUP BY parent_id
        HAVING COUNT(1) > 500 
        ) AS epi500 ON t.id=epi500.parent_id
  ORDER BY epi500.ep_cnt DESC; 
```
**Итоговый вывод**: *5 rows*

|original_title	 |year_of_start	|ep_cnt|
|:---|:---:|:---:|
|Fox & Friends	|1998	|4476|
|Fox & Friends First	|2012	|1750|
|Mr. Squiggle and Friends	|1959	|963|
|Doctor Who	|1963	|695|
|Thomas the Tank Engine & Friends	|1984	|602|

***

### Тренировочная задача 3: 
Напишите SQL-запрос, который выведет оригинальное название *(original_title)*, год начала *(year_of_start)* и количество человек в команде *(com_cnt)*, у которых команда больше 30 человек, отсортировав их по убыванию количества людей. Для этого используйте подзапрос к таблице *principals* с группировкой. Выведите только строки, которые есть в таблице *titles* и в сгруппированной таблице.

```
SELECT titles.original_title,
       titles.year_of_start,
       crew_size.com_cnt
  FROM titles
  JOIN (
        SELECT title_id,
               COUNT(person_id) as com_cnt    
          FROM principals
      GROUP BY title_id
        HAVING COUNT(person_id) > 30
        ) as crew_size ON crew_size.title_id = titles.id 
 ORDER BY crew_size.com_cnt DESC;
```
**Итоговый вывод**: *66 rows*

|original_title	 |year_of_start	|com_cnt|
|:---|:---:|:---:|
|Gliger! Wings of Friendship!/The Gluttonous Urimoo at Urayama House!	|2008	|50|
|Star Wars: Age of Republic: Heroes	|2020	|48|
|Forging Forest Friendships!	|2014	|44|
|Al Pacino and the Chipmunks/That's What Super Friends Are For	|2012	|40|
|Unfair-Weather Friends	|2004	|40|
|...	|...	|...|
|Star Wars: Galactic Battlegrounds	|2001	|32|
|Ray Sharkey/Jack Bruce & Friends	|1981	|32|
|Test Friends	|2021	|32|
|Ring of Friendship	|1984	|32|
|My Little Pony: Friendship Is Magic	|2010	|32|
|...	|...	|...|
|Star Wars: Skywalker Strikes: Complete Volume	|2020	|31|
|Star Wars: Clone Wars: Volume One	|2005	|31|
|The Wednesday Play	|1964	|31|
|George Washington: Cherry Tree Chopper/Star Wars Earn Stripes	|2013	|31|
|Thomas & Friends: Thomas' Holiday Collection	|2017	|31|

***

### Тренировочная задача 4: 
Напишите SQL-запрос, который выведет:
* количество строк в таблице *titles* (назовите поле *title_cnt*);
* количество уникальных записей в поле type таблицы *titles* (назовите поле  *type_cnt*);
* общее количество голосов vote_cnt в таблице *ratings* (назовите поле *sum_vote*).

```
SELECT COUNT(1) AS title_cnt,
       COUNT(DISTINCT titles.type) AS type_cnt,
       (SELECT SUM(ratings.vote_cnt) 
          FROM ratings) AS sum_vote
  FROM titles;      
```
**Итоговый вывод**: *2 rows*

|title_cnt	 |type_cnt	|sum_vote|
|:---|:---:|:---:|	
|53,688.0	|10	|26,850,262	|

***

### Тренировочная задача 5: 
Напишите SQL-запрос, который с помощью коррелированного подзапроса выведет:
* *id* и *original_title* из таблицы *titles*;
* *avg_rating* из таблицы *ratings*, относящийся к строке из *titles*.
Результат отсортируйте по возрастанию *id, original_title, avg_rating*.

```
SELECT id,
       original_title,
       (SELECT ratings.avg_rating
          FROM ratings
         WHERE ratings.title_id = titles.id)
  FROM titles
 ORDER BY 1,2,3;
```
**Итоговый вывод**: *53688 rows*

|id	 |original_title	|avg_rating|
|:---|:---|:---:|
|772	|The Test of Friendship	|5.8|
|2197	|Friends	|5.5|
|2270	|An Indian's Friendship	|None|
|3454	|Three Friends	|6|
|3524	|A Welded Friendship	|None|
|...	|...	|...|
|32408470	|Top 20 Best Friends Episodes	|None|
|32408775	|Rings Of Power DESTROYED Again, Assassin's Creed Shadows CRINGE Gets Slammed, Star Wars Fail!	|None|
|32409125	|Night Lights	|None|
|32409342	|Star Wars Backlash GROWS: Acolyte Angers Last Remaining Fans! The Pro Show Sunday Special	|None|
|32409653	|Wednesday, February 14, 2024	|None|

***

### Тренировочная задача 6: 
Напишите SQL-запрос, который с помощью подзапроса в блоке WHERE выведет строку таблицы *titles*, у которой максимальный рейтинг (*avg_rating* из таблицы *ratings*) и максимальное количество голосов *(vote_cnt)* среди других произведений с тем же рейтингом.

```
SELECT *
  FROM titles
 WHERE id IN (SELECT out.title_id
                FROM ratings AS out
               WHERE out.avg_rating = (SELECT MAX(inr.avg_rating)
                                         FROM ratings AS inr)
               ORDER BY out.vote_cnt DESC
               LIMIT 1); 

```
**Итоговый вывод**: *1 row*

|id	|type	|popular_title	|original_title	|is_adult	|year_of_start	|year_of_end	|run_time	|genres|
|:---|:---|:---|:---|:---|:---|:---|:---|:---|
|2301451	|tvEpisode	|Ozymandias	|Ozymandias	|False	|2013	|None	|47	|Crime,Drama,Thriller|

***

### Тренировочная задача 7: 
Напишите SQL-запрос, который выведет название (*original_title*) произведения с максимальной продолжительностью. Если таких произведений несколько, то выведите все.

```
SELECT original_title
  FROM titles
 WHERE run_time = (SELECT MAX(run_time) FROM titles); 
```
**Итоговый вывод**: *1 row*

|original_title	 |
|:---|
|La Media Inglesa: Directo Extensible|

***

### Тренировочная задача 8: 
Напишите SQL-запрос, который выведет названия *(original_title)* произведений, продолжительностью между *floor* и *ceil* средней продолжительности произведений в таблице *titles*.

Результат отсортируйте по возрастанию *original_title*.

```
SELECT original_title
  FROM titles
 WHERE run_time >= (SELECT FLOOR(AVG(titles.run_time))
                      FROM titles)
   AND run_time <= (SELECT CEIL(AVG(titles.run_time))
                      FROM titles)
 ORDER BY original_title;   
```
**Итоговый вывод**: *83 rows*

|original_title |
|:---|
|An Extraordinary Journey|
|Best Friends Forever?|
|Best Friends Share Two BBCs|
|Brandi Loves Her Father's Friends|
|Bütün Meyhaneleri Istanbul'un|
|...|
|Undyne and Us Are Best Friends Now? | Undertale Pacifist Run|
|Was I Friends with All Your Bullies?|
|Wednesday Flea Market?|
|Wednesday: Morticia Addams - A Porn Parody|
|We want Star Wars questions - Who Wants to be a Millionaire|


***

### Тренировочная задача 9: 
Выберите имена (без фамилий) из таблицы *persons*, которых в таблице *persons* больше 1000. В качестве ответа напишите SQL-запрос, который выведет строки из таблицы *persons* про людей с такими именами.

Результат отсортируйте по возрастанию *id*

```
SELECT *
  FROM persons AS out
 WHERE SPLIT_PART(out.name, ' ', 1) in (SELECT SPLIT_PART(inr.name, ' ', 1)
                                          FROM persons AS inr
                                         GROUP BY SPLIT_PART(inr.name, ' ', 1) 
                                        HAVING COUNT(1) > 1000);
```
**Итоговый вывод**: *3225  rows*

|id	|name	|year_of_birth	|year_of_death	|professions|
|:---|:---|:---:|:---:|:---|
|24	|John Gielgud	|1904	|2000	|actor,director,writer|
|78	|John Wayne	|1907	|1979	|actor,producer,art_department|
|92	|John Cleese	|1939	|nan	|actor,writer,producer|
|135	|John Denver	|1943	|1997	|music_artist,music_department,actor|
|140	|Michael Douglas	|1944	|nan	|actor,producer,director|
|...	|...	|...	|...	|...|
|15866568	|David Tennant	|nan	|nan	|actor|
|15867156	|Michael Atkins	|nan	|nan	|actor|
|15938619	|John Wilson	|nan	|nan	|\N|
|16055621	|David Oladunjoye	|nan	|nan	|actor|
|16136024	|John Ker	|nan	|nan	|cinematographer|

***

### Тренировочная задача 10: 
Напишите SQL-запрос, который выведет тип *(type)*, оригинальное название *(original_title)*, год выпуска *(year_of_start)* и жанр *(genres)* произведений для взрослых *(is_adult)*, отсортировав их по убыванию рейтинга *(avg_rating в таблице ratings)*. Для этого используйте коррелированный подзапрос в секции сортировки.

```
SELECT titles.type,
       titles.original_title,
       titles.year_of_start,
       titles.genres
  FROM titles
 WHERE titles.is_adult = TRUE 
 ORDER BY (SELECT ratings.avg_rating
             FROM ratings
            WHERE ratings.title_id = titles.id);
```
**Итоговый вывод**: *737 rows*

|type	|original_title	|year_of_start	|genres|
|:---|:---|:---:|:---|
|tvEpisode	|Episode #1.5	|2,014	|Adult|
|short	|Star Wars: One Sith-XXX Parody Online	|2,016	|Adult,Short|
|tvEpisode	|Two Best Friends Share a Big Black Cock	|2,014	|Adult,Drama|
|tvEpisode	|Two Friends Enjoy BBC Together	|2,015	|Adult,Drama|
|movie	|Taboo Confessions: Sharing Family with Friends	|2,016	|Adult|
|...	|...	|...	|...|
|movie	|More Than Friends	|1,973	|Adult,Comedy|
|video	|The Adventures of Super 18'' and His Friends the 'C' Men	|1,999	|Adult|
|video	|Old Friends	|2,012	|Adult|
|video	|Best Friends, Better Lovers	|2,020	|Adult,Romance|
|video	|Colin's Sleazy Friends Rocks	|2,002	|Adult,Documentary|


***

### Тренировочная задача 11: 
Напишите SQL-запрос, который выведет имена людей *(поле name из таблицы persons)*, которые судя по таблице *principals* участвовали в создании 10 самых высокооценённых произведений (максимум голосов среди тех, кто набрал максимальный рейтинг *(avg_rating)* из таблицы *ratings*). Итоговый результат отсортируйте по *name*.

```
SELECT DISTINCT per.name
  FROM persons AS per
  LEFT JOIN principals AS pri ON per.id = pri.person_id
 WHERE pri.title_id in (SELECT title_id
                          FROM ratings
                         ORDER BY avg_rating DESC, vote_cnt DESC
                         LIMIT 10)
 ORDER BY per.name
```
**Итоговый вывод**: *61 rows*

|name|
|:---|
|Aaron Paul|
|Adrianna Costa|
|Agast Annand|
|Amarnath Jha|
|Andrea Corbi Fein|
|...|
|Kinshuk Vaidya|
|Laura Fraser|
|Lee Schneller|
|Mark S. Freeborn|
|Melissa Rothschild|
|...|
|Vinod Kumar Singh|
|Vivek Mushran|
|Yashraj Kushwaha|
|Yuzuru Hanyu|
|Zhan Xiao|

***